# **💁🏻🗨️💁🏻‍♂️안개 예측 EDA code**
> **안개량 예측** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최대 10명이 참여할 수 있는 기상청 주관 날씨 빅데이터 경진대회 입니다.     
> 주어진 데이터를 활용하여 안개 상태의 구간을 예측할 수 있는 모델을 만드는 것이 목표입니다!

# Contents  
  
- 필요한 라이브러리 설치  
- 데이터 불러오기  
- 추가적인 파생변수 가져오기

### 1. 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [41]:
# basic
import os, random
import pandas as pd
import numpy as np

# graph
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 폰트
plt.rcParams['font.family'] = 'NanumSquare'

# 마이너스 출력
matplotlib.rcParams['axes.unicode_minus'] = False

In [42]:
# random seed 고정하기
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

### 2. 데이터 불러오기
- 제공된 데이터를 불러옵니다.

> - year : 년도
> - month : 월
> - day : 일
> - hour : 시간
> - minute : 분(10분 단위)
> - stn_id : 지점 번호
> - ws10_deg : 10분 평균 풍향, deg
> - ws10_ms : 10분 평균 풍속, m/s
> - ta : 1분 평균 기온 10분 주기, 섭씨
> - re : 강수 유무 0:무강수, 1:강수
> - hm : 1분 평균 상대 습도 10분 주기, %
> - sun10 : 1분 일사량 10분 단위 합계, MJ
> - ts : 1분 평균 지면온도 10분 주기, 섭씨

- test 없는 데이터 값
> - vis1 : 1분 평균 시정 10분 주기, m
> - class : 시정 구간

시정 구간은 다음과 같다.
- 0초과 200미만 : 1
- 200이상 500미만 : 2
- 500이상 1000미만 : 3
- 1000이상 : 4
- 4번은 맞춰도 스코어가 증가하진 않지만 틀리면 감점

In [43]:
# load makes data
train_knn = pd.read_csv('../data/train_knnimputer.csv')
test_knn = pd.read_csv('../data/test_knnimputer.csv')

### 3. 추가 파생변수

#### 3-1) 시간적 사이클

- 계절에 따른 차이가 존재할 수 있기 때문에 계절에 관한 사이클을 만들어 주도록 하자
- 하루 단위로 온도의 변화가 존재하기 때문에 이를 학습시키기 위해 하루 단위를 변화하도록 하자

In [44]:
# 시간 사이클 변수
train_knn['sin_time'] = np.sin(2 * np.pi * train_knn['time'] / 24)
train_knn['cos_time'] = np.cos(2 * np.pi * train_knn['time'] / 24)

test_knn['sin_time'] = np.sin(2 * np.pi * test_knn['time'] / 24)
test_knn['cos_time'] = np.cos(2 * np.pi * test_knn['time'] / 24)

In [45]:
# 계절 사이클 변수 - 월별 주기
train_knn['sin_month'] = np.sin(2 * np.pi * train_knn['month'] / 12)
train_knn['cos_month'] = np.cos(2 * np.pi * train_knn['month'] / 12)

test_knn['sin_month'] = np.sin(2 * np.pi * test_knn['month'] / 12)
test_knn['cos_month'] = np.cos(2 * np.pi * test_knn['month'] / 12)

#### 3-2) 이상치 대치

- 클래스가 뾰족하게 4 -> 1 -> 4 와 같이 분포하고 있는 것들이 몇개 있다.
- 이를 보완하기 위해 다음과 같은 과정을 진행한다.
- 단, 1000 이하를 중심으로 맞춰야하는 대회이기 때문에 class를 기준으로 적용하였다.

In [46]:
# 지역별로 따로따로 적용하자.
for c in train_knn['stn_id'].unique():

    # 지역별로 잘라서 이상치 -> 결측치 만들기
    tmp = train_knn[train_knn['stn_id'] == c]

    # lag 만들어서 지난번  class 확인하기
    tmp['shift_left_class'] = tmp['class'].shift(1)

    # 이전과의 차이 구하기
    tmp['diff_left_class'] = tmp['class'] - tmp['shift_left_class']

    # lag 만들어서 이다음 class 확인하기
    tmp['shift_right_class'] = tmp['class'].shift(-1)

    # 이다음과의 차이 구하기
    tmp['diff_right_class'] = tmp['class'] - tmp['shift_right_class']

    # 인덱스 뽑기
    idx = tmp[(abs(tmp['diff_left_class']) >= 2) & (abs(tmp['diff_right_class']) >= 2)].index

    # 인덱스 중 가만히 둬야할 인덱스 제외
    new_idx = []
    for i in idx:
        if i-1 not in idx:
            new_idx.append(i)

    # 인덱스로 처리
    new_idx = pd.Index(new_idx)

    # 뾰족한 데이터 결측치로 처리하기
    train_knn['class'].iloc[new_idx] = tmp['shift_left_class'][new_idx]
    train_knn['vis1'].iloc[new_idx] = train_knn['vis1'][new_idx-1]

#### 3-3) 이슬점 온도

- 안개 생기는 기준점을 미리 만들어두자
- 이슬점은 지면온도와 기온을 고려했을 때 안개가 생성되게 하는 가장 좋은 기준점이다.

In [47]:
# 습도가 0인 값 수동으로 채워주기
train_knn['hm'][2893505] = 66.3

In [48]:
# Magnus 공식 상수
a = 17.27
b = 237.7

# 알파 값 계산
train_knn['alpha'] = (a * train_knn['ta']) / (b + train_knn['ta']) + np.log(train_knn['hm'] / 100.0)
test_knn['alpha'] = (a * test_knn['ta']) / (b + test_knn['ta']) + np.log(test_knn['hm'] / 100.0)

# 이슬점온도 계산
train_knn['dew_point'] = (b * train_knn['alpha']) / (a - train_knn['alpha'])
test_knn['dew_point'] = (b * test_knn['alpha']) / (a - test_knn['alpha'])

#### 3-4) 안개 발생 조건

- 다음 생성할 변수는 세가지 영역으로 나눠서 형성할 것이다.

> 1. 안개 생성 조건: 지역별로 생성되는 조건이 상이하기 때문에 안개별로 조건을 달리하여 적용할 것이다.
> 2. 안개 지속 조건: 얼마나 지속될지, 언제까지 지속되는 조건인지에 따라 계속해서 발생하고 있을 수 있기 때문에 적용한다. 조건이 같기 때문에 통일된 변수로 제작한다.
> 3. 안개 짙어짐 조건: 마찬가지로 조건은 거의 동일하다 보면 된다. 

- 안개 종류에 따른 지형
    - 복사
    - 이류
    - 활승
    - 증발

In [49]:
# 온도조건 미리 계산하기
train_knn['diff_air-dew'] = train_knn['ta'] - train_knn['dew_point']
train_knn['diff_ts-dew'] = train_knn['ts'] - train_knn['dew_point']

test_knn['diff_air-dew'] = test_knn['ta'] - test_knn['dew_point']
test_knn['diff_ts-dew'] = test_knn['ts'] - test_knn['dew_point']

In [50]:
train_knn[(train_knn['re'] == 1) & (train_knn['class'] < 4)]

,class,day,hm,minute,month,re,stn_id,sun10,ta,time,...,hour,DateTime,alpha,dew_point,sin_time,cos_time,sin_month,cos_month,diff_air-dew,diff_ts-dew
2648,3,19.0,80.7,30.0,1.0,1,AA,0.00,1.0,9.0,...,9.0,2020-01-19 09:30,-0.142081,-1.939616,7.071068e-01,-0.707107,0.500000,8.660254e-01,2.939616,1.639616
6168,3,12.0,95.1,10.0,2.0,1,AA,0.00,8.1,20.0,...,20.0,2020-02-12 20:10,0.518868,7.362779,-8.660254e-01,0.500000,0.866025,5.000000e-01,0.737221,-0.562779
6611,3,15.0,93.4,0.0,2.0,1,AA,0.00,7.5,22.0,...,22.0,2020-02-15 22:00,0.459963,6.504049,-5.000000e-01,0.866025,0.866025,5.000000e-01,0.995951,0.795951
6699,3,16.0,88.1,40.0,2.0,1,AA,0.02,-1.7,12.0,...,12.0,2020-02-16 12:40,-0.251100,-3.406551,1.224647e-16,-1.000000,0.866025,5.000000e-01,1.706551,5.506551
12397,3,27.0,96.2,20.0,3.0,1,AA,0.00,13.6,2.0,...,2.0,2020-03-27 02:20,0.895887,13.005429,5.000000e-01,0.866025,1.000000,6.123234e-17,0.594571,-0.205429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139634,2,6.0,91.7,50.0,9.0,1,EC,0.00,24.3,3.0,...,3.0,2022-09-06 03:50,1.515112,22.859068,7.071068e-01,0.707107,-1.000000,-1.836970e-16,1.440932,0.340932
3149425,3,13.0,96.7,40.0,11.0,1,EC,0.00,18.6,3.0,...,3.0,2022-11-13 03:40,1.219748,18.064145,7.071068e-01,0.707107,-0.500000,8.660254e-01,0.535855,0.135855
3149426,2,13.0,96.8,50.0,11.0,1,EC,0.00,18.5,3.0,...,3.0,2022-11-13 03:50,1.214530,17.981022,7.071068e-01,0.707107,-0.500000,8.660254e-01,0.518978,0.218978
3149427,2,13.0,96.8,0.0,11.0,1,EC,0.00,18.7,4.0,...,4.0,2022-11-13 04:00,1.227028,18.180212,8.660254e-01,0.500000,-0.500000,8.660254e-01,0.519788,-0.080212


복사안개

In [51]:
# 안개 생성 조건 1. 복사안개
# train
cri = [
    (train_knn['ground'].isin(['A', 'B'])) & # 내륙산간지방
    (train_knn['diff_ts-dew'] <= 2) & # 지면온도와 이슬점의 차이가 2이하
    (train_knn['hm'] >= 85) & # 습도 85% 이상
    (train_knn['ws10_ms'] <= 2.5) & # 풍속 2.5m/s 이하
    (train_knn['re'] == 0) & # 강수 없음
    (train_knn['sun10'] == 0) & # 일사 없음
    (train_knn['month'].isin([9, 10, 11, 12, 1, 2, 3])) # 가을, 겨울, 애매한3월
]

con = [
    1
]

train_knn['make_copyfog'] = np.select(cri, con, default = 0)

In [52]:
# 안개 생성 조건 1. 복사안개
# test
cri = [
    (test_knn['ground'].isin(['A', 'B'])) & # 내륙산간지방
    (test_knn['diff_ts-dew'] <= 2) & # 지면온도와 이슬점의 차이가 2이하
    (test_knn['hm'] >= 85) & # 습도 85% 이상
    (test_knn['ws10_ms'] <= 2.5) & # 풍속 2.5m/s 이하
    (test_knn['re'] == 0) & # 강수 없음
    (test_knn['sun10'] == 0) & # 일사 없음
    (test_knn['month'].isin([9, 10, 11, 12, 1, 2, 3])) # 가을, 겨울, 애매한3월
]

con = [
    1
]

test_knn['make_copyfog'] = np.select(cri, con, default = 0)

이류안개

In [53]:
# 안개 생성 조건 2. 이류안개
# train
cri = [
    (train_knn['ground'].isin(['D', 'E'])) & # 해안가 지역
    (train_knn['diff_ts-dew'] <= 2) & # 지면온도와 이슬점의 차이가 2이하
    (train_knn['hm'] >= 85) & # 습도 85%이상
    (train_knn['ws10_ms'] >= 4) & # 풍속 4m/s이상
    (train_knn['month'].isin([3, 4, 5, 6, 7, 8, 9, 10, 11])) # 봄  ~ 가을
]

con = [
    1
]

train_knn['make_irufog'] = np.select(cri, con, default = 0)

In [54]:
# 안개 생성 조건 2. 이류안개
# test
cri = [
    (test_knn['ground'].isin(['D', 'E'])) & # 해안가 지역
    (test_knn['diff_ts-dew'] <= 2) & # 지면온도와 이슬점의 차이가 2이하
    (test_knn['hm'] >= 85) & # 습도 85%이상
    (test_knn['ws10_ms'] >= 4) & # 풍속 4m/s이상
    (test_knn['month'].isin([3, 4, 5, 6, 7, 8, 9, 10, 11])) # 봄  ~ 가을
]

con = [
    1
]

test_knn['make_irufog'] = np.select(cri, con, default = 0)

활승안개

In [55]:
# 안개 생성 조건 3. 활승안개
# train
cri = [
    (train_knn['ground'].isin(['B', 'C'])) & # 산간 및 동해안
    (train_knn['diff_ts-dew'] <= 2) & # 지면온도와 이슬점의 차이가 2이하
    (train_knn['hm'] >= 85) & # 습도 85%이상
    (train_knn['ws10_ms'] >= 4) & # 풍속 4이상
    (train_knn['re'] == 0)
]

con = [
    1
]

train_knn['make_mountfog'] = np.select(cri, con, default = 0)

In [56]:
# 안개 생성 조건 3. 복사안개
# test
cri = [
    (test_knn['ground'].isin(['B', 'C'])) & # 산간 및 동해안
    (test_knn['diff_ts-dew'] <= 2) & # 지면온도와 이슬점의 차이가 2이하
    (test_knn['hm'] >= 85) & # 습도 85%이상
    (test_knn['ws10_ms'] >= 4) & # 풍속 4이상
    (test_knn['re'] == 0)
]

con = [
    1
]

test_knn['make_mountfog'] = np.select(cri, con, default = 0)

증발안개

In [57]:
# 안개 생성 조건 4. 증발안개
# train
cri = [
    (train_knn['ground'].isin(['A', 'B'])) & # 산간지방
    (train_knn['diff_ts-dew'] <= 2) & # 지면온도와 이슬점의 차이가 2이하
    (train_knn['hm'] >= 90) & # 습도 85%이상
    (train_knn['ws10_ms'] <= 2.5) & # 풍속 2.5 이하
    (train_knn['re'] == 0) & # 날씨가 흐리지 않게 조절
    (train_knn['month'].isin([12, 1, 2, 3]))
]

con = [
    1
]

train_knn['make_gimfog'] = np.select(cri, con, default = 0)

In [58]:
# 안개 생성 조건 4. 증발안개
# test
cri = [
    (test_knn['ground'].isin(['A', 'B'])) & # 산간지방
    (test_knn['diff_ts-dew'] <= 2) & # 지면온도와 이슬점의 차이가 2이하
    (test_knn['hm'] >= 90) & # 습도 85%이상
    (test_knn['ws10_ms'] <= 2.5) & # 풍속 2.5 이하
    (test_knn['re'] == 0) & # 날씨가 흐리지 않게 조절
    (test_knn['month'].isin([12, 1, 2, 3]))
]

con = [
    1
]

test_knn['make_gimfog'] = np.select(cri, con, default = 0)

지속조건

In [59]:
# 안개 지속 조건 
cri = [
    (train_knn['diff_air-dew'] <= 2) & # 기온이 이슬점보다 같거나 낮게
    (train_knn['hm'] >= 90) & # 습도 90%이상
    (train_knn['ws10_ms'] <= 2) & # 풍속 2 이하
    (train_knn['re'] == 0) & # 날씨가 흐리지 않게 조절
    (train_knn['sun10'] <= 0.2) # 일사량 0.2MJ 이하
]

con = [
    1
]

train_knn['retain_fog'] = np.select(cri, con, default = 0)

In [60]:
# 안개 지속 조건 
# test
cri = [
    (test_knn['diff_air-dew'] <= 2) & # 기온이 이슬점보다 같거나 낮게
    (test_knn['hm'] >= 90) & # 습도 90%이상
    (test_knn['ws10_ms'] <= 2) & # 풍속 2 이하
    (test_knn['re'] == 0) & # 날씨가 흐리지 않게 조절
    (test_knn['sun10'] <= 0.2) # 일사량 0.2MJ 이하
]

con = [
    1
]

test_knn['retain_fog'] = np.select(cri, con, default = 0)

가속조건

In [61]:
# 안개 가속 조건 
cri = [
    (train_knn['diff_air-dew'] <= 1) & # 기온이 이슬점보다 같거나 낮게
    (train_knn['hm'] >= 95) & # 습도 95%이상
    (train_knn['ws10_ms'] <= 1) & # 풍속 1 이하
    (train_knn['sun10'] <= 0.2) # 일사량 0.2MJ 이하
]

con = [
    1
]

train_knn['upclass_fog'] = np.select(cri, con, default = 0)

In [62]:
# 안개 가속 조건 
# test
cri = [
    (test_knn['diff_air-dew'] <= 1) & # 기온이 이슬점보다 같거나 낮게
    (test_knn['hm'] >= 95) & # 습도 95%이상
    (test_knn['ws10_ms'] <= 1) & # 풍속 1 이하
    (test_knn['sun10'] <= 0.2) # 일사량 0.2MJ 이하
]

con = [
    1
]

test_knn['upclass_fog'] = np.select(cri, con, default = 0)

In [63]:
for c in ['make_copyfog', 'make_irufog',
       'make_mountfog', 'make_gimfog', 'retain_fog', 'upclass_fog']:
    
    display(pd.crosstab(train_knn[c], train_knn['class']))

class,1,2,3,4
make_copyfog,,,,
0,6614,10491,10985,3045091
1,1073,1101,1222,79883


class,1,2,3,4
make_irufog,,,,
0,7523,11448,11873,3105089
1,164,144,334,19885


class,1,2,3,4
make_mountfog,,,,
0,7684,11589,12197,3120483
1,3,3,10,4491


class,1,2,3,4
make_gimfog,,,,
0,6861,10884,11301,3100244
1,826,708,906,24730


class,1,2,3,4
retain_fog,,,,
0,2298,3566,6889,2820985
1,5389,8026,5318,303989


class,1,2,3,4
upclass_fog,,,,
0,4289,6360,8521,3003306
1,3398,5232,3686,121668


#### 3-5) AWS안개 생성 위험군 분류 지표

- AWS에서 사용하고 있는 기준을 가져와 우리 데이터에 맞게 적용시켜보고자 함
- 실제로 사용되고 있는 방법인 만큼 큰 효과 있기를 기대하며 생성하는 변수

In [64]:
# 안개 위험군 분류 지표
# train
cri = [
    # 5단계: high risk
    (train_knn['hm'] >= 97) & (train_knn['ws10_ms'] <= 7) & (train_knn['re'] == 0),

    # 4단계: middle risk
    (train_knn['hm'] < 97) & (train_knn['hm'] >= 95) & (train_knn['ws10_ms'] <= 7) & (train_knn['re'] == 0),

    # 3단계: Low risk
    (train_knn['hm'] < 95) & (train_knn['hm'] >= 90) & (train_knn['ws10_ms'] <= 7) & (train_knn['re'] == 0),

    # 2단계: Risk not estimates
    (train_knn['hm'] >= 90)
]

con = [
    4, 3, 2, 1
]

train_knn['fog_risk'] = np.select(cri, con, default = 0)

In [65]:
# 안개 위험군 분류 지표
# test
cri = [
    # 5단계: high risk
    (test_knn['hm'] >= 97) & (test_knn['ws10_ms'] <= 7) & (test_knn['re'] == 0),

    # 4단계: middle risk
    (test_knn['hm'] < 97) & (test_knn['hm'] >= 95) & (test_knn['ws10_ms'] <= 7) & (test_knn['re'] == 0),

    # 3단계: Low risk
    (test_knn['hm'] < 95) & (test_knn['hm'] >= 90) & (test_knn['ws10_ms'] <= 7) & (test_knn['re'] == 0),

    # 2단계: Risk not estimates
    (test_knn['hm'] >= 90)
]

con = [
    4, 3, 2, 1
]

test_knn['fog_risk'] = np.select(cri, con, default = 0)

In [66]:
# 연도 원상태로 되돌리기
cri = [
    train_knn['year'] == 2020,
    train_knn['year'] == 2021,
    train_knn['year'] == 2022,
    train_knn['year'] == '2020.0',
    train_knn['year'] == '2021.0',
    train_knn['year'] == '2022.0',
    train_knn['year'] == 'I',
    train_knn['year'] == 'J',
    train_knn['year'] == 'K',
]

con = [
    'I', 'J', 'K', 'I', 'J', 'K', 'I', 'J', 'K'
]
train_knn['year'] = np.select(cri, con, default = 2022)

In [67]:
# save the data
train_knn.to_csv('../data/train_preprocessed_data.csv', index = False)
test_knn.to_csv('../data/test_preprocessed_data.csv', index = False)